In [ ]:
import lzma
import json
import xml.etree.cElementTree as ET

import pandas as pd
pd.options.display.max_colwidth = 200
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pprint

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import gensim

import warnings
warnings.filterwarnings(action='ignore')

In [47]:
import gensim

Loading the Data

In [11]:
# Illinois dataset
path = r'C:\Users\ilyaa\Desktop\data\data.jsonl.xz'

In [12]:
# a list to hold the sampled cases 
cases = []
max_cases = 10000

In [13]:
with lzma.open(path) as inline:
    for i, line in enumerate(inline):
        record = json.loads(str(line, 'utf-8'))
        cases.append(record)
        if i == max_cases - 1:
            print("Number of Cases:", len(cases))
            break

Number of Cases: 10000


In [17]:
# XML parsing function
def ET_parse_xml(case):
    feat_dict = {}
    header = True
    feat_dict["court"] = case['court']['name']
    feat_dict["date"] = case['decision_date']
    for elem in ET.fromstring(case['casebody']['data']):
        opinions = []
        if elem.tag.split("}")[1] == "opinion":
            op = {}
            text = []
            op["type"] = elem.attrib["type"]
            op["author"] = ""
            for opinion_element in elem.getchildren():
                if opinion_element.tag.split("}")[1] == 'author':
                    op["author"] = opinion_element.text.replace(u'\xad', '')
                else:
                    text.append(opinion_element.text.replace(u'\xad', ''))
            op["text"] = " ".join(text)
            opinions.append(op)
    feat_dict["opinions"] = opinions
    return feat_dict

In [18]:
# For each file, parse with parse_xml() and add to list of parsed files
parsed_files = []
for case in cases:
    doc = ET_parse_xml(case) 
    parsed_files.append(doc)
   

Divide into 2 periods

In [20]:
# Create a pandas dataframe to visualize our dataset
raw_df = pd.DataFrame(parsed_files)

# Parse dates
raw_df["date"] = pd.to_datetime(raw_df["date"])

# Strip out some sparse early data
df = raw_df[raw_df["date"] >= '1820-01-01']

In [61]:
df.head()

,court,date,opinions
0,Illinois Appellate Court,2003-05-02,"[{'type': 'majority', 'author': 'PRESIDING JUSTICE McDADE', 'text': 'delivered the opinion of the court: Michael Ratliff-El filed a complaint for mandamus relief (735 ILCS 5/14 — 101 et seq. (West..."
1,Illinois Appellate Court,2003-05-01,"[{'type': 'majority', 'author': 'JUSTICE MAAG', 'text': 'delivered the opinion of the court: The plaintiffs, Amanda and Brittany Adams, by their father and next friend, Tommy Adams, and Tommy Adam..."
2,Illinois Appellate Court,2003-04-25,"[{'type': 'majority', 'author': 'PRESIDING JUSTICE HOPKINS', 'text': 'delivered the opinion of the court: The petitioner, Alisha Bowlby, appeals the trial court’s judgment dissolving her marriage ..."
3,Illinois Appellate Court,2003-04-08,"[{'type': 'majority', 'author': 'JUSTICE MAAG', 'text': 'delivered the opinion of the court: C. Michael Witters and Diane Witters (plaintiffs), individually and derivatively on behalf of Midwest T..."
4,Illinois Appellate Court,2003-03-20,"[{'type': 'majority', 'author': 'PRESIDING JUSTICE THEIS', 'text': 'delivered the opinion of the court: Defendant, Blockbuster, Inc. BACKGROUND On February 18, 1999, plaintiffs filed a class-actio..."


word2vec model

In [32]:
raw_documents = df.opinions.tolist()
custom_stop_words = [""]

In [40]:
import re
class TokenGenerator:
    def __init__( self, documents, stopwords ):
        self.documents = documents
        self.stopwords = stopwords
        self.tokenizer = re.compile(r"\S+") 

    def __iter__( self ):
        print("Word2Vec model ...")
        for doc in self.documents:
            tokens = []
            for tok in self.tokenizer.findall( doc ):
                if tok in self.stopwords:
                    tokens.append( "<stopword>" )
                elif len(tok) >= 2: 
                    tokens.append( tok )
            yield tokens

In [41]:
docgen = TokenGenerator( raw_documents, custom_stop_words )


In [ ]:
from gensim.models import Word2Vec

In [63]:
print(docgen)

In [62]:
model = Word2Vec(docgen, 
                 size=300, 
                 window=4, 
                 sg = 1,
                 min_count=1, 
                 workers=4)

Word2Vec model ...


TypeError: expected string or bytes-like object